# Lab 9-3: MNIST nn Xavier - Xavier initialization 를 이용한 신경망으로 MNIST 데이터셋 분류하기

In [ ]:
#import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform= transforms.ToTensor(),
                         download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform= transforms.ToTensor(),
                         download= True)

#dataset loader
data_loader = torch.utils.data.DataLoader(dataset= mnist_train,
                                          batch_size= batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
#nn layers
linear1 = torch.nn.Linear(784, 256, bias= True)
linear2 = torch.nn.Linear(256, 256, bias= True)
linear3 = torch.nn.Linear(256, 10, bias= True)
relu = torch.nn.ReLU()

#xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

#model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) #Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [ ]:
#train
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  print('Epoch: {:2d} Cost= {:.9f}'.format(epoch + 1, avg_cost))
print('Learning finished')

Epoch:  1 Cost= 0.246763036
Epoch:  2 Cost= 0.093066528
Epoch:  3 Cost= 0.061003156
Epoch:  4 Cost= 0.043708097
Epoch:  5 Cost= 0.032775719
Epoch:  6 Cost= 0.025442224
Epoch:  7 Cost= 0.021619247
Epoch:  8 Cost= 0.018301070
Epoch:  9 Cost= 0.015056375
Epoch: 10 Cost= 0.013457117
Epoch: 11 Cost= 0.013505858
Epoch: 12 Cost= 0.012631116
Epoch: 13 Cost= 0.010484267
Epoch: 14 Cost= 0.011389917
Epoch: 15 Cost= 0.008104536
Learning finished


In [ ]:
#test
with torch.no_grad():
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9765999913215637
Label: 5
Prediction: 5
